In [ ]:
# (images) train, val, test = 4070, 452, 949
# (labels) train, val, test = 4070, 452, 949

import os
my_path = "/content/drive/MyDrive/yolo_data_v1/labels/val"
f = 0
for filename in os.listdir(my_path):
    if filename.endswith('.txt'):
        f += 1
print(f)

452


In [ ]:
# (images) train, val, test = 3663, 407, 452
# (labels) train, val, test = 3663, 407, 452

my_path_ = "/content/drive/MyDrive/yolo_data/images/test"
ff = 0
for filename in os.listdir(my_path_):
    if filename.endswith('.png'):
        ff += 1
print(ff)

452


In [ ]:
!pip install torch torchvision torchaudio
!pip install pyyaml
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import cv2
import os
import torch
import torchvision
import yaml

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def convert_to_yolo_format(row, img_width, img_height):
    x_center = (row['x_min'] + row['x_max']) / 2 / img_width
    y_center = (row['y_min'] + row['y_max']) / 2 / img_height
    width = (row['x_max'] - row['x_min']) / img_width
    height = (row['y_max'] - row['y_min']) / img_height
    return f"{row['class_id']} {x_center} {y_center} {width} {height}"

In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/VinDr/annotations/resized_train_annotations.csv')
# df = df[df['class_name'] != 'No finding']
# df['class_id'].unique()
# Load the dataset
df_init = pd.read_csv('/content/drive/MyDrive/VinDr/annotations/resized_train_annotations.csv')  # Replace with your dataset file name

# Separate "No finding" cases and other cases
no_finding_df = df_init[df_init['class_name'] == 'No finding']
other_cases_df = df_init[df_init['class_name'] != 'No finding']

# Sample approximately 500 "No finding" cases
no_finding_sample = no_finding_df.sample(n=500, random_state=42)

# Combine the sampled "No finding" cases with the other cases
df = pd.concat([no_finding_sample, other_cases_df])

# Shuffle the final dataset
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Display unique class IDs
print(df['class_name'].unique())


['Pleural effusion' 'Other lesion' 'Emphysema' 'Cardiomegaly'
 'Pulmonary fibrosis' 'Pleural thickening' 'Aortic enlargement'
 'Nodule/Mass' 'Infiltration' 'ILD' 'Lung Opacity' 'Rib fracture' 'Edema'
 'No finding' 'Pneumothorax' 'Mediastinal shift' 'Calcification'
 'Enlarged PA' 'Consolidation' 'Atelectasis' 'Clavicle fracture'
 'Lung cavity' 'Lung cyst']


In [ ]:
df[df['class_name']=='No finding']['image_id'].count()

500

In [ ]:
# Load the dataset
df_test = pd.read_csv('/content/drive/MyDrive/VinDr/annotations/resized_test_annotations.csv')  # Replace with your dataset file name
df_test['class_id'].unique()

array([ 1,  2, 10,  5, 12,  8,  9,  7,  4,  3, 11, 15, 16,  6, 21, 18, 19,
       20, 13, 14, 17])

In [ ]:
# set to YOLO Format
# Load the dataset

# Path to images and annotations
image_dir = "/content/drive/MyDrive/resized_train_images"
yolo_labels_dir = "/content/drive/MyDrive/yolo_labels"

if not os.path.exists(yolo_labels_dir):
    os.makedirs(yolo_labels_dir)

# Process each image and generate YOLO labels
for image_id in df['image_id'].unique():

    image_path = os.path.join(image_dir, image_id)
    image_data = df[df['image_id'] == image_id]


    img_width, img_height = 1024, 1024

    # Generate YOLO annotations
    yolo_annotations = []
    for _, row in image_data.iterrows():


      if not pd.isnull(row['x_min']):  # Skip rows without bounding box
          yolo_annotations.append(convert_to_yolo_format(row, img_width, img_height))

    # Save YOLO annotation file
    label_file = os.path.join(yolo_labels_dir, f"{os.path.splitext(image_id)[0]}.txt")
    with open(label_file, 'w') as f:
        f.write("\n".join(yolo_annotations))

In [ ]:
files = 0
for filename in os.listdir(yolo_labels_dir):
    if filename.endswith('.txt'):
        files += 1
print(files)

5022


In [ ]:
# Path to images and annotations
image_dir_test = "/content/drive/MyDrive/resized_test_images"  # Replace with your images directory
yolo_labels_dir_test = "/content/drive/MyDrive/yolo_labels_test"

if not os.path.exists(yolo_labels_dir_test):
    os.makedirs(yolo_labels_dir_test)

# Process each image and generate YOLO labels
for image_id in df_test['image_id'].unique():

    image_path = os.path.join(image_dir_test, image_id)
    image_data = df_test[df_test['image_id'] == image_id]

    # Assuming all images have the same dimensions, replace with actual dimensions
    img_width, img_height = 1024, 1024

    # Generate YOLO annotations
    yolo_annotations_test = []
    for _, row in image_data.iterrows():

      if not pd.isnull(row['x_min']):  # Skip rows without bounding box
          yolo_annotations_test.append(convert_to_yolo_format(row, img_width, img_height))

    # Save YOLO annotation file
    label_file_test = os.path.join(yolo_labels_dir_test, f"{os.path.splitext(image_id)[0]}.txt")
    with open(label_file_test, 'w') as f:
        f.write("\n".join(yolo_annotations_test))

In [ ]:
files_test = 0
for filename in os.listdir(yolo_labels_dir_test):
    if filename.endswith('.txt'):
        files_test += 1
print(files_test)

949


In [ ]:
# Move files to their respective directories
def move_files(image_files, image_dir, label_dir, target_image_dir, target_label_dir):
    for image_file in image_files:
        shutil.copy(os.path.join(image_dir, image_file), os.path.join(target_image_dir, image_file))
        label_file = os.path.splitext(image_file)[0] + ".txt"
        shutil.copy(os.path.join(label_dir, label_file), os.path.join(target_label_dir, label_file))

In [ ]:
# Function to get valid images with non-empty labels
def get_valid_images(image_dir, label_dir):

    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
    valid_images = []
    for image_file in image_files:

        label_file = os.path.splitext(image_file)[0] + ".txt"
        label_path = os.path.join(label_dir, label_file)
        if os.path.exists(label_path):  # Check if the label file exists
            with open(label_path, "r") as f:
                if f.read().strip():  # Keep only non-empty label files
                    valid_images.append(image_file)

    return valid_images

In [ ]:
# import random
# import shutil

# # Paths
# train_image_dir = "/content/drive/MyDrive/resized_train_images"  # Directory for train images
# # test_image_dir = "/content/drive/MyDrive/split_data/images/val"    # Directory for test images
# label_dir = "/content/drive/MyDrive/yolo_labels"
# # label_dir_test = "/content/drive/MyDrive/split_data/labels/val"  # Directory containing all labels
# output_dir = "/content/drive/MyDrive/yolo_data"         # Output directory

# # Create directories for splits
# train_dir = os.path.join(output_dir, "images/train")
# val_dir = os.path.join(output_dir, "images/val")
# test_dir = os.path.join(output_dir, "images/test")

# train_label_dir = os.path.join(output_dir, "labels/train")
# val_label_dir = os.path.join(output_dir, "labels/val")
# test_label_dir = os.path.join(output_dir, "labels/test")

# os.makedirs(train_dir, exist_ok=True)
# os.makedirs(val_dir, exist_ok=True)
# os.makedirs(test_dir, exist_ok=True)
# os.makedirs(train_label_dir, exist_ok=True)
# os.makedirs(val_label_dir, exist_ok=True)
# os.makedirs(test_label_dir, exist_ok=True)

# # Parameters for validation split
# val_split_ratio = 0.1  # 10% of train data for validation

# # Get valid train and test images
# train_images = get_valid_images(train_image_dir, label_dir)
# # test_images = get_valid_images(test_image_dir, label_dir_test)

# # Shuffle the train images for splitting
# random.shuffle(train_images)

# # Calculate validation size
# val_count = int(len(train_images) * val_split_ratio)

# # Split train into train and validation
# val_files = train_images[:val_count]
# train_files = train_images[val_count:]

# # Move train, validation, and test files
# move_files(train_files, train_image_dir, label_dir, train_dir, train_label_dir)
# move_files(val_files, train_image_dir, label_dir, val_dir, val_label_dir)
# move_files(test_images, test_image_dir, label_dir_test, test_dir, test_label_dir)

# print("Dataset successfully split into train, val, and test sets, excluding 'No findings' files!")

KeyboardInterrupt: 

In [ ]:
import random
import shutil

# Paths
train_image_dir = "/content/drive/MyDrive/resized_train_images"  # Directory for train images
label_dir = "/content/drive/MyDrive/yolo_labels"
output_dir = "/content/drive/MyDrive/yolo_data"         # Output directory

# Create directories for splits
train_dir = os.path.join(output_dir, "images/train")
val_dir = os.path.join(output_dir, "images/val")
test_dir = os.path.join(output_dir, "images/test")

train_label_dir = os.path.join(output_dir, "labels/train")
val_label_dir = os.path.join(output_dir, "labels/val")
test_label_dir = os.path.join(output_dir, "labels/test")

def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Ensure all directories exist
for directory in [train_dir, val_dir, test_dir, train_label_dir, val_label_dir, test_label_dir]:
    create_directory(directory)

# Parameters for splitting
test_split_ratio = 0.1  # 10% of train data for testing
val_split_ratio = 0.1   # 10% of remaining train data for validation

# Get valid train images
train_images = get_valid_images(train_image_dir, label_dir)
random.shuffle(train_images)

# Split train images into train, val, and test
num_test = int(len(train_images) * test_split_ratio)
test_files = train_images[:num_test]
remaining_files = train_images[num_test:]

num_val = int(len(remaining_files) * val_split_ratio)
val_files = remaining_files[:num_val]
train_files = remaining_files[num_val:]

# Move files
move_files(train_files, train_image_dir, label_dir, train_dir, train_label_dir)
move_files(val_files, train_image_dir, label_dir, val_dir, val_label_dir)
move_files(test_files, train_image_dir, label_dir, test_dir, test_label_dir)

print("Dataset successfully split into train, val, and test sets!")


Dataset successfully split into train, val, and test sets!


In [ ]:
# Create a dictionary mapping ClassName to ClassID
name_to_id_mapping = dict(zip(df['class_name'], df['class_id']))


# Display the mappings
name_to_id_mapping


{'Pleural effusion': 11,
 'Other lesion': 10,
 'Emphysema': 19,
 'Cardiomegaly': 8,
 'Pulmonary fibrosis': 5,
 'Pleural thickening': 6,
 'Aortic enlargement': 7,
 'Nodule/Mass': 4,
 'Infiltration': 1,
 'ILD': 9,
 'Lung Opacity': 2,
 'Rib fracture': 21,
 'Edema': 22,
 'No finding': 0,
 'Pneumothorax': 18,
 'Mediastinal shift': 16,
 'Calcification': 12,
 'Enlarged PA': 13,
 'Consolidation': 3,
 'Atelectasis': 15,
 'Clavicle fracture': 20,
 'Lung cavity': 14,
 'Lung cyst': 17}

In [ ]:
(df_test['image_id']=="d523d9a02febab7fd910c61e0ad00480").sum()

0

In [ ]:
from ultralytics import YOLO

dataset_yaml = """
train: /content/drive/MyDrive/yolo_data/images/train
val: /content/drive/MyDrive/yolo_data/images/val
test: /content/drive/MyDrive/yolo_data/images/test

nc: 23

names:
  - No finding
  - Infiltration
  - Lung Opacity
  - Consolidation
  - Nodule/Mass
  - Pulmonary fibrosis
  - Pleural thickening
  - Aortic enlargement
  - Cardiomegaly
  - ILD
  - Other lesion
  - Pleural effusion
  - Calcification
  - Enlarged PA
  - Lung cavity
  - Atelectasis
  - Mediastinal shift
  - Lung cyst
  - Pneumothorax
  - Emphysema
  - Clavicle fracture
  - Rib fracture
  - Edema
"""

# Save dataset.yaml to split_data directory
dataset_yaml_path = "/content/drive/MyDrive/yolo_data/dataset.yaml"

with open(dataset_yaml_path, "w") as f:
    f.write(dataset_yaml)

print(f"Dataset YAML saved at: {dataset_yaml_path}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Dataset YAML saved at: /content/drive/MyDrive/yolo_data/dataset.yaml


In [ ]:
from ultralytics import YOLO

# Train YOLOv8s

In [ ]:
# Download YOLOv8 Pretrained Weights

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
model = YOLO('/content/yolov8s.pt')  # select version of YOLO

# Fine-Tune the Model
model.train(
    data='/content/drive/MyDrive/yolo_data/dataset.yaml',  # Path to the dataset YAML file
    epochs=20,            # Number of training epochs
    imgsz=1024,            # Input image size
    batch= 4,             # Batch size
    # device='cpu' # GPU device index
    device='cuda'
)

# Evaluate the Fine-Tuned Model
metrics = model.val(data='/content/drive/MyDrive/yolo_data/dataset.yaml')
print(metrics)

Ultralytics 8.3.87 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/content/yolov8s.pt, data=/content/drive/MyDrive/yolo_data/dataset.yaml, epochs=20, time=None, patience=100, batch=4, imgsz=1024, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tru

train: Scanning /content/drive/MyDrive/yolo_data/labels/train.cache... 3663 images, 0 backgrounds, 1 corrupt: 100%|██████████| 3663/3663 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/yolo_data/images/train/cffa67149b86b1359623d25f061a2063.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0815]
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/yolo_data/labels/val.cache... 407 images, 0 backgrounds, 0 corrupt: 100%|██████████| 407/407 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00037, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 1024 train, 1024 val
Using 2 dataloader workers
Logging results to runs/detect/train2
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


20 epochs completed in 1.950 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 22.6MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 22.6MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.87 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,134,485 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00:19<00:00,  2.60it/s]


                   all        407       2307      0.535      0.242      0.265      0.127
          Infiltration         57         84      0.522      0.286      0.316       0.14
          Lung Opacity        116        182      0.447      0.319      0.293      0.122
         Consolidation         35         45      0.459      0.244      0.313      0.144
           Nodule/Mass         81        193      0.562      0.316      0.331      0.161
    Pulmonary fibrosis        135        273      0.398      0.315      0.303       0.12
    Pleural thickening        180        363      0.428      0.231      0.249     0.0871
    Aortic enlargement        276        320      0.727      0.847      0.866       0.51
          Cardiomegaly        219        230      0.739      0.909      0.917      0.559
                   ILD         36         66      0.438      0.303      0.347      0.188
          Other lesion        115        187      0.156      0.128     0.0824     0.0336
      Pleural effusio

val: Scanning /content/drive/MyDrive/yolo_data/labels/val.cache... 407 images, 0 backgrounds, 0 corrupt: 100%|██████████| 407/407 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 102/102 [00:23<00:00,  4.41it/s]


                   all        407       2307      0.529      0.242      0.265      0.126
          Infiltration         57         84      0.505      0.298      0.314      0.139
          Lung Opacity        116        182      0.437      0.313      0.289      0.122
         Consolidation         35         45      0.455      0.244      0.308      0.143
           Nodule/Mass         81        193      0.542      0.306      0.318      0.161
    Pulmonary fibrosis        135        273      0.395      0.326      0.305      0.119
    Pleural thickening        180        363      0.424      0.231      0.251      0.087
    Aortic enlargement        276        320      0.723      0.847      0.866      0.508
          Cardiomegaly        219        230      0.738      0.909      0.919      0.561
                   ILD         36         66      0.446      0.303      0.347      0.187
          Other lesion        115        187      0.154      0.128     0.0829      0.034
      Pleural effusio

In [ ]:
import shutil
from google.colab import files

# Zip the 'runs' folder located in '/content/'
shutil.make_archive('/content/runs', 'zip', '/content/runs')

# Download the zipped file
files.download('/content/runs.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Test YOLOv8s

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 932.9/932.9 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
!yolo task=detect mode=predict model=best.pt conf=0.1 source=/content/drive/MyDrive/VinDr/test_png/5c55f871429730b84a8ee275839b8aae.png \
    save=True project=/content/test_results name=single_image_test

Ultralytics 8.3.88 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,134,485 parameters, 0 gradients, 28.5 GFLOPs

image 1/1 /content/drive/MyDrive/VinDr/test_png/5c55f871429730b84a8ee275839b8aae.png: 1024x832 2 Aortic enlargements, 1 Cardiomegaly, 41.0ms
Speed: 9.9ms preprocess, 41.0ms inference, 138.1ms postprocess per image at shape (1, 3, 1024, 832)
Results saved to /content/test_results/single_image_test5
💡 Learn more at https://docs.ultralytics.com/modes/predict


# Integration of Random Forest and YOLOv8s

In [ ]:
!pip install ultralytics

In [ ]:
!pip install scikit-learn==1.2.2

In [ ]:
import torch
import numpy as np
import joblib
from ultralytics import YOLO
import pandas as pd

In [ ]:
# Load trained YOLOv8s model
yolo_model = YOLO("/content/drive/MyDrive/yolo_v8s.pt")  # Replace with your trained YOLO model path
print("YOLOv8s Loaded!")

YOLOv8s Loaded!


In [ ]:
# Load trained Random Forest Classifier
rf_model = joblib.load("/content/drive/MyDrive/multi_label_random_forest.pkl")  # Replace with your trained RF model path
print("Random Forest Classifier Loaded!")

Random Forest Classifier Loaded!


In [ ]:
# Define the 22 local labels used as features in RF training
local_labels = [
    "Aortic enlargement", "Atelectasis", "Calcification", "Cardiomegaly",
    "Clavicle fracture", "Consolidation", "Edema", "Emphysema",
    "Enlarged PA", "ILD", "Infiltration", "Lung Opacity",
    "Lung cavity", "Lung cyst", "Mediastinal shift", "Nodule/Mass",
    "Pleural effusion", "Pleural thickening", "Pneumothorax",
    "Pulmonary fibrosis", "Rib fracture", "Other lesion"
]

In [ ]:
# Define the 6 global disease labels (target variables)
global_labels = ["COPD", "Lung tumor", "Pneumonia", "Tuberculosis", "Other diseases", "No finding"]

In [ ]:
def preprocess_yolo_output(detections, label_map):
    """
    Converts YOLO detection results into a binary feature vector.
    """
    feature_vector = np.zeros(len(label_map), dtype=int)
    for det in detections:
        class_id = int(det[5])  # Assuming class ID is at index 5 in YOLO output
        if class_id < len(label_map):  # Ensure class_id is within range
            feature_vector[class_id] = 1  # Mark detected class as 1
    return feature_vector

In [ ]:
def predict_global_disease(image_path):
    """
    Runs YOLOv8 on the image, extracts local features, and predicts the global disease.
    """
    # Run YOLO inference
    results = yolo_model(image_path)

    # Extract detections (YOLO format: [x1, y1, x2, y2, confidence, class_id])
    detections = results[0].boxes.data.cpu().numpy() if results[0].boxes is not None else []

    # Convert YOLO output into a feature vector
    local_features = preprocess_yolo_output(detections, local_labels)

    # Predict global disease using the Random Forest Classifier
    global_prediction = rf_model.predict([local_features])

    # Get the predicted disease label
    predicted_disease = global_labels[np.argmax(global_prediction)]

    return predicted_disease

In [ ]:
# Example usage 1
image_path = "/content/drive/MyDrive/VinDr/test_png/5c55f871429730b84a8ee275839b8aae.png"  # Replace with actual image path
global_disease = predict_global_disease(image_path)
print("Predicted Global Disease:", global_disease)


image 1/1 /content/drive/MyDrive/VinDr/test_png/5c55f871429730b84a8ee275839b8aae.png: 1024x832 1 Aortic enlargement, 44.6ms
Speed: 24.1ms preprocess, 44.6ms inference, 379.0ms postprocess per image at shape (1, 3, 1024, 832)
Predicted Global Disease: Other diseases


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X

In [ ]:
# Example usage 2
image_path = "/content/drive/MyDrive/VinDr/test_png/b4e78fa47b3675c8de1ceae320bc4bf9.png"  # Replace with actual image path
global_disease = predict_global_disease(image_path)
print("Predicted Global Disease:", global_disease)


image 1/1 /content/drive/MyDrive/VinDr/test_png/b4e78fa47b3675c8de1ceae320bc4bf9.png: 992x1024 1 Aortic enlargement, 1 Cardiomegaly, 44.8ms
Speed: 8.7ms preprocess, 44.8ms inference, 2.6ms postprocess per image at shape (1, 3, 992, 1024)
Predicted Global Disease: Other diseases


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X

In [ ]:
# def preprocess_yolo_output(detections, label_map):
#     """
#     Converts YOLO detection results into a binary feature vector.
#     """
#     feature_vector = np.zeros(len(label_map), dtype=int)
#     for det in detections:
#         class_id = int(det[5])  # Assuming class ID is at index 5 in YOLO output
#         if class_id < len(label_map):  # Ensure class_id is within range
#             feature_vector[class_id] = 1  # Mark detected class as 1
#     return feature_vector

# def predict_global_disease(image_path):
#     """
#     Runs YOLOv8 on the image, extracts local features, and predicts the global disease.
#     """
#     # Run YOLO inference
#     results = yolo_model(image_path)

#     # Extract detections (YOLO format: [x1, y1, x2, y2, confidence, class_id])
#     detections = results[0].boxes.data.cpu().numpy() if results[0].boxes is not None else []

#     # Convert YOLO output into a feature vector
#     local_features = preprocess_yolo_output(detections, local_labels)

#     # Predict global disease using the Random Forest Classifier
#     global_prediction = rf_model.predict([local_features])

#     return global_prediction[0]

# # Example usage
# image_path = "test_xray.jpg"  # Replace with actual image path
# global_disease = predict_global_disease(image_path)
# print("Predicted Global Disease:", global_disease)

In [ ]:
# Load trained YOLOv8s model
yolo_model_n = YOLO("/content/drive/MyDrive/yolo_v8n.pt")  # Replace with your trained YOLO model path
print("YOLOv8n Loaded!")

YOLOv8n Loaded!


In [ ]:
def predict_global_disease_(image_path):
    """
    Runs YOLOv8 on the image, extracts local features, and predicts the global disease.
    """
    # Run YOLO inference
    results = yolo_model_n(image_path)

    # Extract detections (YOLO format: [x1, y1, x2, y2, confidence, class_id])
    detections = results[0].boxes.data.cpu().numpy() if results[0].boxes is not None else []

    # Convert YOLO output into a feature vector
    local_features = preprocess_yolo_output(detections, local_labels)

    # Predict global disease using the Random Forest Classifier
    global_prediction = rf_model.predict([local_features])

    # Get the predicted disease label
    predicted_disease = global_labels[np.argmax(global_prediction)]

    return predicted_disease

In [ ]:
# Example usage 1
image_path = "/content/drive/MyDrive/VinDr/test_png/5c55f871429730b84a8ee275839b8aae.png"  # Replace with actual image path
global_disease = predict_global_disease_(image_path)
print("Predicted Global Disease:", global_disease)


image 1/1 /content/drive/MyDrive/VinDr/test_png/5c55f871429730b84a8ee275839b8aae.png: 1024x832 1 Aortic enlargement, 1 Cardiomegaly, 31.9ms
Speed: 10.8ms preprocess, 31.9ms inference, 1.7ms postprocess per image at shape (1, 3, 1024, 832)
Predicted Global Disease: Other diseases


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X

In [ ]:
# Example usage 2
image_path = "/content/drive/MyDrive/VinDr/test_png/b4e78fa47b3675c8de1ceae320bc4bf9.png"  # Replace with actual image path
global_disease = predict_global_disease_(image_path)
print("Predicted Global Disease:", global_disease)


image 1/1 /content/drive/MyDrive/VinDr/test_png/b4e78fa47b3675c8de1ceae320bc4bf9.png: 992x1024 1 Aortic enlargement, 1 Cardiomegaly, 74.9ms
Speed: 9.3ms preprocess, 74.9ms inference, 2.6ms postprocess per image at shape (1, 3, 992, 1024)
Predicted Global Disease: Other diseases


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X

In [ ]:
aa15cfcfca7605465ca0513902738b95

In [1]:
# prompt: write code for downloading this image from path /content/drive/MyDrive/VinDr/test_png/

from google.colab import files

image_path = "/content/drive/MyDrive/VinDr/test_png/aa15cfcfca7605465ca0513902738b95.png" #@param {type:"string"}
try:
  files.download(image_path)
except FileNotFoundError:
  print(f"Error: Image not found at {image_path}")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>